In [1]:
#Luckily, being in the case where the system has no self-interaction, we can verify that the Schmidt states do converge to the pointer states. The pointer states will correspond to eigenstates of the interaction Hamiltonian.
